In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

from cnn_model import Model
from data_loader import movie_data

data = movie_data()

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)

/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
learning_rate = 0.001   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
keep_prob = 0.8
##################################################


model = Model(use_clip=use_clip, class_size=class_size, learning_rate=learning_rate, keep_prob=keep_prob)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

AttributeError: 'Model' object has no attribute 'conv'

In [ ]:
def test_model():
    num_it = 10
    test_loss, test_cnt, test_right = 0, 0, .0

    for _ in range(num_it):
        test_x, test_label = data.get_test(BATCH_SIZE)
        loss, out = sess.run([model.loss, model.y_pred],
                              feed_dict={model.x_image: test_x, model.y_label: test_label, keep_prob:1.0})
        test_loss += loss
        test_cnt += 1
        for i, o in enumerate(out):
            if o == test_label[i]:
                test_right += 1
    print(" * test loss: {:.3f} | acc: {:.3f}\n".format(test_loss / test_cnt, test_right / test_cnt / BATCH_SIZE))
    
avg_loss, it_cnt, same = 0, 0, .0
it_log, it_test, it_save, it_sample = 10, 100, 1000, 100
start_time = time.time()
for it in range(0, 10000):
    train_x, label = data.get_train(BATCH_SIZE)
#     for img in train_x:
#         print(img.shape)
#     print('-----')
#     for l in label:
#         print(l)
    loss, _,  out = sess.run([model.loss, model.update, model.y_pred],
                            feed_dict={model.x_image: train_x, model.y_label: label})
#     print(prob)
    for i, o in enumerate(out):
        if o == label[i]:
            same += 1
    avg_loss += loss
    it_cnt += 1
    if it % it_log == 0 and it:
        print(" it: {:4d} | loss: {:.3f} | acc: {:.3f} - {:.2f}s".format(
            it, avg_loss / it_cnt, same/BATCH_SIZE/it_log, time.time() - start_time))
        avg_loss, it_cnt, same = 0, 0, .0
        
    if it % it_test == 0 and it > 0:
        test_model()